### All Imports

In [1]:
import pandas as pd 
import sqlite3 as sq 

### Create General Class

In [223]:


class AsigbeyStore():
    # def __init__(self):
    #     pass 

    # def create_database(self):
    def __init__(self):
        connection = sq.connect('asigbey_database.db')
        connection.execute(""" CREATE TABLE IF NOT EXISTS parent_databse 
                                (
                                    id                 INTEGER PRIMARY KEY AUTOINCREMENT,
                                    date_register      CURRENT_TIMESTAMP, 
                                    firstname          TEXT NOT NULL, 
                                    surname            TEXT NOT NULL, 
                                    phone              INT  NOT NULL, 
                                    email              CHAR(120), 
                                    date_of_birth      DATETIME,     
                                    password1          TEXT NOT NULL
                                );  
                            """
                           )
        
        connection.execute(""" CREATE TABLE IF NOT EXISTS medicine_statatics
                                (
                                    id                 INTEGER PRIMARY KEY AUTOINCREMENT,
                                    medicine_name      TEXT NOT NULL, 
                                    code               TEXT NOT NULL, 
                                    quantity           TEXT NOT NULL, 
                                    buying_price       INT NOT NULL, 
                                    selling_price      INT NOT NULL,  
                                    profit             INT NOT NULL,   
                                    expiring_date      TEXT NOT NULL
                                );  
                            """
                    )
        
        connection.execute(""" CREATE TABLE IF NOT EXISTS user_data 
                                (
                                    id                 INTEGER PRIMARY KEY AUTOINCREMENT,
                                    date_register      CURRENT_TIMESTAMP, 
                                    firstname          TEXT NOT NULL, 
                                    surname            TEXT NOT NULL, 
                                    phone              TEXT NOT NULL,     
                                    password           TEXT NOT NULL
                                );  
                            """
                    )
        

        connection.execute(""" CREATE TABLE IF NOT EXISTS transaction_history  
                                (
                                    id                 INTEGER PRIMARY KEY AUTOINCREMENT,
                                    date_register      CURRENT_TIMESTAMP, 
                                    sold_by            TEXT NOT NULL, 
                                    customer           TEXT NOT NULL, 
                                    medicine           TEXT NOT NULL,     
                                    quantity           TEXT NOT NULL, 
                                    profit_per_sale    TEXT NOT NULL, 
                                    total_profit       TEXT NOT NULL
                                );  
                            """
                    )       
        
        connection.close()

        print("Table created successfully")
        # pass


#  ============   DASH BOARD   ================================================

    def dash_board(self):
        pass 


#  ============   MEDICINE TYPE   ================================================

    def medicine_type(self):
        pass 


#  ============   MEDICINE VOLUME   ================================================

    def medicine_volume(self):
        pass 



#   ============================   ADD USER    ================================================

    def add_user(self, firstname, surname, phone, password):
        text = "Could not signup: Check your internet connection, or Try later"
        connection = sq.connect('asigbey_database.db')
        all_users = pd.read_sql_query("SELECT * FROM user_data", connection)
        
        users_firstname = [firstname.lower().strip() for firstname in all_users['firstname']]
        users_surname   = [surname.lower().strip() for surname in all_users['surname']]

        firstname = firstname.lower().strip()
        surname   = surname.lower().strip()

        if (firstname in users_firstname) and (surname in users_surname):
            print("IN")
            text = (f"{firstname.capitalize()} {surname.capitalize()} is already in the a User added")

        else: 
            print("NOT IN")
            connection = sq.connect("asigbey_database.db")
            connection.execute(f""" INSERT INTO user_data (date_register, firstname,     surname,     phone,     password) VALUES 
                                     (CURRENT_TIMESTAMP, '                 {firstname}', '{surname}', '{phone}', '{password}')""")    
            connection.commit()
            connection.close()
            text = (f"{firstname.capitalize()} {surname.capitalize()} has successfully been added ")
        return text 
    



#   ============================   USER LOGIN    ================================================

    def user_login(self, surname, password):
        connection = sq.connect('asigbey_database.db')
        all_users = pd.read_sql_query(f"SELECT * FROM user_data WHERE surname='{surname}' AND password='{password}'", connection) 

        if len(all_users) > 0:
            print("LOGIN SUCCESSFUL")
        else:
            print("LOGIN FAILED.... Wrong Password OR You are not in the Database....  Ask the Administrator to add you...")
        return 
    


#  ========================   SHOW USER INFORMATION  ================================================

    def show_users_information(self, user_firstname="all"):
        if user_firstname == "all":
            connection = sq.connect("asigbey_database.db")
            user_table = pd.read_sql_query(f"SELECT * FROM user_data", connection)
            # user = '*'
        else:
            user = user_firstname
            connection = sq.connect("asigbey_database.db")
            user_table = pd.read_sql_query(f"SELECT * FROM user_data WHERE firstname='{user}'", connection)
        return user_table
    



#   ============================   ADD MEDICINE     ================================================

    def add_medicine(self, medicine_name, code, quantity, buying_price, selling_price, expiring_date):
        connection = sq.connect("asigbey_database.db")
        medicine_data = pd.read_sql_query(f"SELECT * FROM medicine_statatics", connection)
        medicine_code_list = [code_.lower().strip() for code_ in medicine_data['code']]
        code = str(code.lower().strip())

        if code in medicine_code_list:
            print("Yes") 
            medicine_data = pd.read_sql_query(f"SELECT * FROM medicine_statatics WHERE code='{code}'", connection)
            print(medicine_data)

            # previous_medicine_name   = medicine_data["medicine_name"][0]
            # previous_code            = medicine_data["code"][0]
            previous_quantity        = int(medicine_data["quantity"][0]) 
            previous_buying_price    = float(medicine_data["buying_price"][0]) 
            previous_selling_price   = float(medicine_data["selling_price"][0])
            # previous_expiring_date   = medicine_data["expiring_date"][0]

            quantity = quantity + previous_quantity

            if int(buying_price) != previous_buying_price:
                buying_price = buying_price
            
            if int(selling_price) != previous_selling_price:
                selling_price = selling_price
            
            profit_ = (int(selling_price) - int(buying_price)) * int(quantity)
            connection = sq.connect("asigbey_database.db")
            connection.execute(f"UPDATE medicine_statatics SET quantity = '{quantity}', buying_price = '{buying_price}', selling_price = '{selling_price}', profit = '{profit_}'   WHERE code = '{code}';") 
            connection.commit()
            connection.close()

        else:
            print("No")
            profit_ = (int(selling_price) - int(buying_price)) * int(quantity)
            connection = sq.connect("asigbey_database.db")
            connection.execute(f""" INSERT INTO medicine_statatics (medicine_name,     code,     quantity,     buying_price,     selling_price,     profit,     expiring_date)  VALUES 
                                    (                             '{medicine_name}', '{code}', '{quantity}', '{buying_price}', '{selling_price}', '{profit_}', '{expiring_date}')""") 
            connection.commit()
            connection.close()

        print("Medicine added SUCCESSFULLY")



#   ============================   SHOW MEDICINE STATISTICS    =======================================

    def show_medicine_stats(self, medicine_code="all"):
        if medicine_code == "all":
            connection = sq.connect("asigbey_database.db")
            medicine_table = pd.read_sql_query(f"SELECT * FROM medicine_statatics", connection)
        else: 
            connection = sq.connect("asigbey_database.db")
            medicine_table = pd.read_sql_query(f"SELECT * FROM medicine_statatics WHERE code='{medicine_code}'", connection)
        return medicine_table 

    def sales_report(self):
        pass 


#   ============================   EXPIRED/EXPERING MEDICINE    =======================================

    def expired_medicine(self, date):
        connection = sq.connect("asigbey_database.db")
        medicine_table = pd.read_sql_query(f"SELECT * FROM medicine_statatics WHERE code='{date}'", connection)
        return medicine_table



#   ============================   NEW TRANSACTIONS     ================================================

    def new_transaction(self, sold_by, customer, medicine_code, quantity):
        text = ""
        quantity_history = quantity
        connection = sq.connect("asigbey_database.db")
        medicine_data = pd.read_sql_query(f"SELECT * FROM medicine_statatics", connection)
        medicine_code_list = [code_.lower().strip() for code_ in medicine_data['code']]
        code = str(medicine_code.lower().strip())

        if code in medicine_code_list: 
            print("IN")
            medicine_data = pd.read_sql_query(f"SELECT * FROM medicine_statatics WHERE code='{code}'", connection)
            previous_quantity = int(medicine_data["quantity"][0]) 

            if quantity < previous_quantity:
                print("proceed to complete transaction. Update the database")
                previous_buying_price    = float(medicine_data["buying_price"][0]) 
                previous_selling_price   = float(medicine_data["selling_price"][0])

                quantity = previous_quantity - quantity
                buying_price = previous_buying_price
                selling_price = previous_selling_price 
                profit_ = (int(selling_price) - int(buying_price)) * int(quantity)

                connection = sq.connect("asigbey_database.db")
                connection.execute(f"UPDATE medicine_statatics SET quantity = '{quantity}', profit = '{profit_}'   WHERE code = '{code}';") 
                connection.commit()
                connection.close()

                ##### Insert into transaction table 

                connection = sq.connect("asigbey_database.db")
                connection.execute(f""" INSERT INTO transaction_history (date_register,     sold_by,      customer,     medicine,   quantity,    profit_per_sale, total_profit) VALUES 
                                        (                                CURRENT_TIMESTAMP, '{sold_by}', '{customer}', '{code}',   '{quantity_history}', '{profit_}',     '{profit_}')""")    
                connection.commit()
                connection.close()

            if quantity == previous_quantity:
                print("proceed to complete transaction. Delete this drug from store")
 
                connection = sq.connect("asigbey_database.db")
                connection.execute(f"DELETE FROM  medicine_statatics  WHERE code = '{code}';") 
                connection.commit()
                connection.close()

            if quantity > previous_quantity:
                print("this transaction CANNOT be complete. Quantity in store is less than what you intend to sell")
  
        else:
            print("NOT IN")
            text = "You do not have this medicine in the store. If you do have, then consider updating the databse"
            print(text)



#   ==========================   TRANSACTION HISTORY     =======================================

    def transactions_history(self):
        connection = sq.connect("asigbey_database.db")
        medicine_table = pd.read_sql_query(f"SELECT * FROM transaction_history", connection)
        return medicine_table


#   ==========   STOCK VALUE     =======================================
    def stock_value(self):
        pass 


#   ==========  EXPENDITURE     =======================================
    def expenditure(self):
        pass 

#   ==========  BACK UP     =======================================
    def back_up(self):
        pass 


asigbey_store = AsigbeyStore()
print("")
asigbey_store.user_login(surname="zorve", password="2222")
print()


asigbey_store.add_user(firstname="kojo", surname="nsiah", phone=2414555, password=2222)
asigbey_store.show_users_information() 

Table created successfully

LOGIN SUCCESSFUL

IN


,id,date_register,firstname,surname,phone,password
0,1,2023-05-17 01:39:29,name,zorve,0000,1234
1,2,2023-05-17 11:18:31,peter,zorve,2414555,2222
2,3,2023-05-17 11:18:46,peter,antwi,2414555,2222
3,4,2023-05-17 11:45:38,peter1,antwi,2414555,2222
4,5,2023-05-17 11:46:02,pet,antwi,2414555,2222
5,6,2023-05-17 11:47:37,pet,antwii,2414555,2222
6,7,2023-05-17 11:49:31,kojo,nsiah,2414555,2222


### Show Medicine Statistics

In [134]:
asigbey_store = AsigbeyStore()
asigbey_store.show_medicine_stats(medicine_code="all")

Table created successfully


,id,medicine_name,code,quantity,buying_price,selling_price,profit,expiring_date
0,3,nana bitters,nana1,310,80,160,24800,yesterday
1,4,nana bitters,nana2,400,10,20,4000,yesterday
2,5,nana bitters,aaduro,400,10,20,4000,yesterday
3,6,nana bitters,nana,60,80,160,4800,next week
4,7,nana bitters,aduro,510,80,160,40800,next week


### Add Medicine 

In [133]:
asigbey_store = AsigbeyStore()
asigbey_store.add_medicine(medicine_name="nana bitters", code='aduro', quantity=102, buying_price=80, selling_price=160, expiring_date="next week")

Table created successfully
Yes
   id medicine_name   code quantity  buying_price  selling_price  profit  \
0   7  nana bitters  aduro      408            80            160   32640   

  expiring_date  
0     next week  
Medicine added SUCCESSFULLY


### Show User Information

In [22]:
asigbey_store = AsigbeyStore()
asigbey_store.show_users_information(user_firstname="name")

Table created successfully


,id,date_register,firstname,surname,phone,password


### Add User 

In [9]:
add_user = AsigbeyStore()

print(add_user.add_user(firstname="name", surname="zorve", phone="0000", password="1234"))

Table created successfully
Empty DataFrame
Columns: [phone]
Index: []
[]
User added SUCCESSFUL


In [10]:
class Payment():
    def __init__(self, price):
        self.price = price  + (0.25 * price)

    def calculate_discount(self, discount):
        return self.price - (self.price * (discount/100))

    def get_price(self):
        return self.price 

    def final_price(self, discount):
        return  self.calculate_discount(discount)

In [11]:
connection = sq.connect('asigbey_database.db')

connection.execute(""" CREATE TABLE IF NOT EXISTS parent_databse 
                 (
                    id                 INTEGER PRIMARY KEY AUTOINCREMENT,
                    date_register      CURRENT_TIMESTAMP, 
                    firstname          TEXT NOT NULL, 
                    surname            TEXT NOT NULL, 
                    phone              INT  NOT NULL, 
                    email              CHAR(120), 
                    date_of_birth      DATETIME,     
                    password1          TEXT NOT NULL
                 );  """)

print("Table created successfully")

Table created successfully


In [ ]:
location = 'data'
table_name = 'table_name'

def init():    
    global conn
    global c
    conn = sqlite3.connect(location)
    c = conn.cursor()
    create_database()

def create_database():
    sql = 'create table if not exists ' + table_name + ' (id integer)'
    c.execute(sql)
    conn.commit()

def create_database2():
    sql = 'create table ' + table_name + '(id integer)'
    c.execute(sql)
    conn.commit()

def clear_database():
    sql = 'drop table ' + table_name
    c.execute(sql)
    conn.commit()

def remake():
    clear_database()
    create_database() # Replacing this with create_database2() works every time
    insert_record(1)
    conn.commit()

def insert_record(id):
    sql = 'insert into ' + table_name + ' (id) values (%d)' % (id)
    c.execute(sql)
    print 'Inserted ', id

init()
remake()